In [1]:
import json
import os.path
import pandas as pd
import numpy as np
from preprocessing.logParser import openLogs

In [2]:
#path to event logs
BASE_DIR = os.path.abspath('test_statistics.ipynb')
BASE_DIR = os.path.dirname(BASE_DIR)
BASE_DIR = BASE_DIR + '/spbu_ACADRU_spring_2018-TL'
cursor = openLogs(BASE_DIR)

In [3]:
#and questions NOT LIKE '__' means that we needn't questions like {}
#and attempts < 4 means that we use only three first attempts, other attempts are not interesting
request = """
        select course_name,
        attempts, questions,
        page, grade, max_grade
        from problem_check
        where course_name = "course-v1:spbu+ACADRU+spring_2018"
        and questions NOT LIKE '__'
        and attempts < 4
        """.format("https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/b434df438c2045eba63afc089d64b3de/244a81ebf8a64b609cd8587bbe18edc5/")
cursor.execute(request)
data = cursor.fetchall()

columns_names = []
for i in cursor.description[:]:
    columns_names.append(i[0])

df = pd.DataFrame(data=data, columns=columns_names)

### модуль тестов, пункт 1 a: (средний результат за тест и медиана)

In [4]:
#calculating: average grade, median of grade, number of answers on test
avg=df.groupby(['course_name','page','attempts','max_grade'])['grade'].agg([np.mean, np.median, np.size])
avg=avg.reset_index()

In [5]:
avg = avg.rename(columns={'size': 'number_of_solutions'})

### модуль тестов, пункт 1 с: (процент правильных ответов на вопрос)

In [6]:
#parsing questions string in two columns: question and result
listForQuestions=pd.DataFrame(columns=('course_name','page','question','attempts','result','max_grade'))

for rowCounter in range(df.shape[0]):
    currentAttempt=df.attempts[rowCounter]
    currentPage=df.page[rowCounter]
    currentCourse=df.course_name[rowCounter]
    currentGrade=df.max_grade[rowCounter]
    
    row=(df.questions[rowCounter][1:-1]).split(',')
    for cellRow in row:
        tmprow=cellRow.split(':')
        if tmprow[1]=='True':
            listForQuestions=listForQuestions.append({'course_name':currentCourse,'page':currentPage,'question':tmprow[0],'attempts':currentAttempt,'result':1,'max_grade':currentGrade}, ignore_index=True)
        else:
            listForQuestions=listForQuestions.append({'course_name':currentCourse,'page':currentPage,'question':tmprow[0],'attempts':currentAttempt,'result':0,'max_grade':currentGrade}, ignore_index=True)


In [7]:
listForQuestions['question'] = listForQuestions['question'].map(str.strip) #deleting spaces

In [8]:
#calculating: sum of grades on question and number of answers on question
perc=listForQuestions.groupby(['course_name','page', 'attempts','max_grade','question'])['result'].agg([np.sum , np.size])

In [9]:
#calcilating percentage of right answers on each question
perc=100*perc['sum']/perc['size']
perc=perc.reset_index()

In [10]:
#merge questions and percentage of right answers
perc["questions"] = perc["question"].map(str) +":" +perc[0].map(str)
perc=perc.drop(columns=["question", 0])

In [11]:
#merge questions in test
perc=perc.groupby(['course_name','page','attempts','max_grade'])['questions'].apply(','.join).reset_index()
perc["questions"]='{ ' + perc["questions"] +'}'

### Output

In [12]:
#merge results of analysis (average and percentade) into one table
res=pd.merge(avg, perc, on=['course_name','page','attempts','max_grade'], how='outer')

In [13]:
course_name=(res["course_name"][0]+ "_test_statistics.json").replace('+', '_').split(':')[1]

In [14]:
#writing json file
with open(course_name + "_test_statistics.json", "w") as data_file:
    json.dump(res.to_json(orient='records'), data_file, indent=2)

In [15]:
#print(res.to_json(orient='table'))

{"schema": {"fields":[{"name":"index","type":"integer"},{"name":"course_name","type":"string"},{"name":"page","type":"string"},{"name":"attempts","type":"integer"},{"name":"max_grade","type":"number"},{"name":"mean","type":"number"},{"name":"median","type":"number"},{"name":"number_of_solutions","type":"number"},{"name":"questions","type":"string"}],"primaryKey":["index"],"pandas_version":"0.20.0"}, "data": [{"index":0,"course_name":"course-v1:spbu+ACADRU+spring_2018","page":"https:\/\/courses.openedu.ru\/courses\/course-v1:spbu+ACADRU+spring_2018\/xblock\/block-v1:spbu+ACADRU+spring_2018+type@problem+block@062bec6c9167fb11f29c\/handler\/xmodule_handler\/problem_check","attempts":1,"max_grade":1.0,"mean":0.6666666667,"median":1.0,"number_of_solutions":3.0,"questions":"{ \"062bec6c9167fb11f29c_2_1\":66.66666666666667}"},{"index":1,"course_name":"course-v1:spbu+ACADRU+spring_2018","page":"https:\/\/courses.openedu.ru\/courses\/course-v1:spbu+ACADRU+spring_2018\/xblock\/block-v1:spbu+ACAD